# Summary

We're interested in GitHub project health. I first establish a baseline
using known successful projects and projects which were previously successful
but have now been retired. This notebook is intended to get the basic working
thing together then we can scale up later.

## Dependencies

In [1]:
import os

import okra

## Prepare Data

In [3]:
DATA = "/Users/tylerbrown/code/"
os.listdir(DATA)

['.DS_Store', 'linux', 'docker-ce']